In [2]:
# ------- SERVER EXTENSIONS ---------
lib =  r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities/droplet_dataset'
lib2 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities'
lib3 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/data_analysis'
lib4 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy'
lib5 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/scripts'
import sys
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)
from utilities.general_helpers import *
# ------- SERVER EXTENSIONS ---------
import numpy as np
import pandas as pd
import scipy
import sklearn
from sklearn.manifold import TSNE
import pickle
# from Bio.Cluster import kcluster
import os
import numpy as np
import yaml
from os.path import join
import os
import pandas
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sys
import seaborn as sns
# import statsmodels as sm
import scipy.stats as stats
from scipy.stats import rankdata
from sklearn.manifold import TSNE
import pickle
import numpy as np
from utilities.general_helpers import flatten_list
from utilities.ML_environment import find_marker_genes_in_cluster
# import pyclustering
from shutil import copyfile
import matplotlib.pyplot as plt

# Finding a threshold for variance so that we will have 4k~ genes.

##### Loads pickle file of NORMALIZED cohort

In [41]:
# cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/cohort_normalized_26.6.21.pkl'
cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/myeloid_normalized_26.6.21.pkl'
normalized_cohort = pickle.load(open(cohort_path, 'rb'))

##### Run the next block if you want to keep only protein coding genes:

In [42]:
print(f'Number of genes in cohort: {normalized_cohort.number_of_genes}')
normalized_cohort.filter_protein_coding_genes()
print(f'Number of genes in cohort after filtering: {normalized_cohort.number_of_genes}')

Number of genes in cohort: 32738
Dataset was cleared from non-protein coding genes
Number of genes in cohort after filtering: 20273


##### Finds a threshold for all cohort

In [43]:
variances = np.var(normalized_cohort.counts, axis=0)
variances_sorted = np.flip(variances[np.argsort(variances)])

In [44]:
num_requested_genes = 4000
print('The 20 variances around the 4Kth gene:')
print(variances_sorted[num_requested_genes-10: num_requested_genes+10], end='\n\n')
print(f'To get 4K~ genes, you need to take a number between {variances_sorted[num_requested_genes-1: num_requested_genes+1]}')
all_cohort_variance_threshold = variances_sorted[num_requested_genes-1]+ (variances_sorted[num_requested_genes]-variances_sorted[num_requested_genes-1])/2
print(f'You can take: {all_cohort_variance_threshold}')


The 20 variances around the 4Kth gene:
[0.28921257 0.28910463 0.28902773 0.28886497 0.28879879 0.28871017
 0.28868781 0.28868044 0.2886425  0.28861065 0.28853778 0.28846126
 0.28845333 0.28828159 0.2881256  0.2879871  0.28792872 0.28781682
 0.28773422 0.28765617]

To get 4K~ genes, you need to take a number between [0.28861065 0.28853778]
You can take: 0.28857421811408934


##### Finds a threshold for immune cells

In [6]:
immune_cells = normalized_cohort.filter_cells_by_property('is_immune', True)

In [7]:
variances = np.var(immune_cells.counts, axis=0)
variances_sorted = np.flip(variances[np.argsort(variances)])

In [15]:
num_requested_genes = 4000
print('The 20 variances around the 4Kth gene:')
print(variances_sorted[num_requested_genes-10: num_requested_genes+10], end='\n\n')
print(f'To get 4K~ genes, you need to take a number between {variances_sorted[num_requested_genes-1: num_requested_genes+1]}')
print(f'You can take: {variances_sorted[num_requested_genes-1]+ (variances_sorted[num_requested_genes]-variances_sorted[num_requested_genes-1])/2}')

The 20 variances around the 4Kth gene:
[0.31973475 0.31968604 0.31968266 0.31962894 0.31943816 0.3193458
 0.31925898 0.31907132 0.31904501 0.31898028 0.31885335 0.31879423
 0.31868693 0.31862338 0.31858656 0.31857346 0.31850735 0.31842375
 0.31817571 0.31817437]

To get 4K~ genes, you need to take a number between [0.31898028 0.31885335]
You can take: 0.31891681578550196


##### Finds a threshold for tumor cells

In [4]:
tumor_cells = normalized_cohort.filter_cells_by_property('is_cancer', True)


In [5]:
variances = np.var(tumor_cells.counts, axis=0)
variances_sorted = np.flip(variances[np.argsort(variances)])

In [6]:
num_requested_genes = 4000
print('The 20 variances around the 4Kth gene:')
print(variances_sorted[num_requested_genes-10: num_requested_genes+10], end='\n\n')
print(f'To get 4K~ genes, you need to take a number between {variances_sorted[num_requested_genes-1: num_requested_genes+1]}')

The 20 variances around the 4Kth gene:
[0.27538899 0.27535058 0.27527164 0.27523672 0.27522969 0.27518354
 0.27512949 0.27512372 0.275116   0.27510745 0.27507093 0.27488508
 0.27487266 0.27482577 0.27477006 0.27472933 0.27469943 0.27464011
 0.27434547 0.27420142]

To get 4K~ genes, you need to take a number between [0.27510745 0.27507093]


### Saves PKLs of immune, tumor, or all cells with after variance filter

Tumor:

In [7]:
out_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/tumor_cells_4k_genes.pkl'
variance = 0.27508 # [0.27510745 0.27507093]

tumor_filtered = tumor_cells.filter_genes_by_variance(variance)
print(f'Number of genes in file: {tumor_filtered.number_of_genes}')
pickle.dump((tumor_filtered), open(out_path, 'wb'), protocol=4)
print('Done')

Dataset was cleared from genes with variance of less than 0.27508
Done


Immune:

In [18]:
out_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/immune_cells_26.6.21_4k_genes.pkl'
variance = 0.31891681578550196

immune_filtered = immune_cells.filter_genes_by_variance(variance)
print(f'Number of genes in file: {immune_filtered.number_of_genes}')
pickle.dump((immune_filtered), open(out_path, 'wb'), protocol=4)
print('Done')

Dataset was cleared from genes with variance of less than 0.31891681578550196
Number of cells in file: 4000
Done


All cells (cohort):

In [45]:
out_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/myeloid_normalized_26.6.21_4k_genes.pkl'
variance = all_cohort_variance_threshold

cohort_filtered = normalized_cohort.filter_genes_by_variance(variance)
print(f'Number of genes in file: {cohort_filtered.number_of_genes}')
pickle.dump((cohort_filtered), open(out_path, 'wb'), protocol=4)
print('Done')

Dataset was cleared from genes with variance of less than 0.28857421811408934
Number of genes in file: 4000
Done


# Protein coding overlap with high variance genes (4K first genes)

Loads protein coding DF:

In [30]:
PROTEIN_CODING_FILE = r'/storage/md_keren/shitay/Data/tables/gene_ens_map.xlsx'
protein_coding_df = pandas.read_excel(PROTEIN_CODING_FILE, header = None, names=['gene_id', 'gene', '1', '2'])
protein_coding_genes = protein_coding_df[protein_coding_df['2'] == 'protein_coding']['gene'].tolist()

Loads cohort that you want to see the number of protein coding genes out of all genes in the cohort:

In [31]:
cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/cytotoxic_normalized_26.6.21_4k_genes.pkl'
normalized_cohort = pickle.load(open(cohort_path, 'rb'))

In [32]:
overlap_genes = intersection_of_lists(normalized_cohort.gene_names, protein_coding_genes)
print(f'Number of protein coding genes if DF: {len(protein_coding_genes)}')
print(f'Number of genes in cohort: {normalized_cohort.number_of_genes}')
print(f'Number of protein coding genes in cohort: {len(overlap_genes)}')

Number of protein coding genes if DF: 20776
Number of genes in cohort: 4000
Number of protein coding genes in cohort: 4000


# Debug

In [34]:
normalized_cohort.number_of_cells

25815

In [39]:
tumor_cells.number_of_cells
sum(tumor_cells.cells_information.getattr('is_cancer'))

97225

In [44]:
ii = pickle.load(open(r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/immune_cells_4k_genes.pkl', 'rb'))

In [56]:
ii.counts[700:900].max()

10.936111592118117

In [77]:
sum(tumor_filtered.cells_information.getattr('is_cancer'))
tumor_filtered.number_of_cells

97225

In [62]:
cohort_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/cohort_normalized_10.5.21.pkl'
normalized_cohort = pickle.load(open(cohort_path, 'rb'))

In [66]:
save_path = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/immune_cells_10.5.21.pkl'
immune_cells = normalized_cohort.filter_cells_by_property('is_immune', True)

In [70]:
immune_cells.number_of_genes
pickle.dump((immune_cells), open(save_path, 'wb'), protocol=4)

In [7]:
tt = pickle.load(open(out_path, 'rb'))
tt


In [9]:
tt.number_of_genes

4000